<a href="https://colab.research.google.com/github/MarinaZRocha/Mod_Ling_Robotica/blob/main/E02_Mod_ling_179741.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Universidade Federal do Rio Grande (FURG)
### Programa de Pós-Graduação em Computação (PPGComp)
#### Disciplina: ModLing para Robótica 2025
##### Prof. Rodrigo da Silva Guerra
##### E02: Sistema de Recomendação usando Word2Vec

### Descrição do Exercício

Neste exercício, o objetivo é construir um sistema de recomendação de produtos com base no histórico de compras de um e-commerce. A premissa é que a sequência de produtos adquiridos por um cliente em uma única transação possui uma relação semântica. Por exemplo, um cliente que compra itens para uma festa de aniversário provavelmente selecionará produtos relacionados a esse tema. Utilizaremos o algoritmo Word2Vec para capturar essas relações, tratando cada produto como uma "palavra" e cada compra (carrinho de compras) como uma "frase".

Você baixará a base de dados e deverá realizar todo o processo de tratamento, treinamento do modelo e implementação das funcionalidades de recomendação.

### Instruções

O exercício é demonstrado [nessa videoaula](https://youtu.be/PKOGlgVhS_A), mas você é encorajado a tentar implementar por conta própria.

1.  **Carregamento e Pré-processamento dos Dados:**
    * Faça o download do arquivo [`retail.xlsx`](https://drive.google.com/uc?id=1NK-2z0l-qTplDJJ2SHpTVBGRP3zWAK-n).
    * Carregue os dados em um DataFrame do Pandas.
    * Realize a limpeza dos dados, removendo registros com valores nulos.
    * Converta a coluna de códigos de produtos (`StockCode`) para o formato de string.

2.  **Criação do Dicionário de Produtos:**
    * Crie um dicionário Python que mapeie os códigos dos produtos (`StockCode`) às suas respectivas descrições (`Description`). Este dicionário será utilizado para a interpretação dos resultados, permitindo a visualização das descrições dos produtos recomendados.

3.  **Preparação dos Dados para Treinamento:**
    * O histórico de compras de cada cliente deve ser tratado como uma "frase", onde cada produto é uma "palavra". Agrupe os produtos por cada `InvoiceNo`, criando uma lista de listas, onde cada lista interna representa uma compra.

4.  **Treinamento do Modelo Word2Vec:**
    * Utilize a biblioteca `gensim` para treinar um modelo Word2Vec com as "frases" (compras) criadas no passo anterior.

5.  **Desenvolvimento das Funções de Recomendação:**
    * **Função 1: Produtos Similares**
        * Implemente uma função que, ao receber um código de produto, retorne uma lista dos produtos mais similares, com base no espaço latente aprendido pelo Word2Vec. A função deve exibir a descrição do produto de entrada e as descrições dos produtos mais similares, juntamente com seus respectivos scores de similaridade.
    * **Função 2: Recomendação por Carrinho de Compras**
        * Implemente uma função que receba uma lista de códigos de produtos (simulando um carrinho de compras).
        * Calcule o vetor médio de embedding para os produtos presentes no carrinho.
        * Utilize o vetor médio para encontrar os produtos mais próximos no espaço de embedding, que serão as recomendações.
        * A função deve retornar uma lista com as descrições dos produtos recomendados, excluindo aqueles que já estão no carrinho, e seus respectivos scores de similaridade.

### Código Inicial

In [1]:
# Código para baixar os dados
!gdown https://drive.google.com/uc?id=1NK-2z0l-qTplDJJ2SHpTVBGRP3zWAK-n

Downloading...
From: https://drive.google.com/uc?id=1NK-2z0l-qTplDJJ2SHpTVBGRP3zWAK-n
To: /content/retail.xlsx
100% 23.7M/23.7M [00:00<00:00, 36.6MB/s]


In [2]:
import pandas as pd
import numpy as np


In [3]:
df = pd.read_excel('retail.xlsx')


In [4]:
df.dropna(inplace=True)

In [5]:
product_dict = dict(zip(df['StockCode'], df['Description']))

In [6]:
product_dict['10123C']

'HEARTS WRAPPING TAPE '

In [7]:
customers = df['CustomerID'].unique().tolist()
df.drop_duplicates(inplace=True, subset='StockCode', keep='last')
len(customers)

4372

In [ ]:
import random
import numpy as np

np.random.shuffle(customers)

In [20]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [28]:
def client_history(customer_id, infos):

  purchases = []
  #tt_history = infos.groupby(["CustomerID", "InvoiceNo"])['StockCode'].apply(list).reset_index()
  for client in customer_id:
    purchase = infos[infos['CustomerID'] == client]['StockCode'].tolist()
    #purchase = infos[infos['CustomerID'] == client]['StockCode'].tolist()
    purchases.append(purchase)

  #purchases = tt_history.groupby(['CustomerID'])['StockCode'].apply(list).tolist()
  return purchases


In [25]:
train_size = int((0.9*len(customers)))

cl_train = customers[:train_size]
cl_test = customers[train_size:len(customers)]

df_train = df[df['CustomerID'].isin(cl_train)]
df_test = df[df['CustomerID'].isin(cl_test)]

In [29]:
purchase_train = client_history(cl_train, df_train)
purchase_test = client_history(cl_test, df_test)

In [30]:
len(purchase_train)

3934

In [32]:
purchase_train[20]

[20713,
 21931,
 22553,
 20724,
 20719,
 20726,
 22328,
 20712,
 22326,
 21481,
 '85099B',
 22356,
 20728,
 'POST']

In [ ]:
import gensim
model = gensim.models.Word2Vec(orders_train, vector_size=50, window=5, workers=10, epochs=10)

In [ ]:
#from gensim.models import Word2Vec

In [ ]:
def similar_products(product_id):

  list_similar = model.wv.most_similar(product_id)
  print(f'Similar products to {product_dict[product_id]}:')
  for product in list_similar:
    print(f'{product_dict[product]}')
